In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import pandas as pd
import time
import os
from tqdm import tqdm

In [3]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
IMAGE_SIZE = (150, 150)

In [4]:
# Paths to dataset
DATA_DIR = "/kaggle/input/intel-image-classification/seg_train/seg_train"
VAL_DIR = "/kaggle/input/intel-image-classification/seg_test/seg_test"

# Preprocessing
transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = datasets.ImageFolder(DATA_DIR, transform=transform)
val_dataset = datasets.ImageFolder(VAL_DIR, transform=transform)

# Create dataloaders
BATCH_SIZE = 64
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

In [5]:
# Basic ResNet Block
class BasicBlock(nn.Module):
    expansion = 1  # For BasicBlock, expansion is always 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        if self.downsample:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)
        return out

In [6]:
# ResNet18 Model
class ResNet18(nn.Module):
    def __init__(self, num_classes=6):
        super().__init__()

        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(BasicBlock, 64, 2, stride=1)
        self.layer2 = self._make_layer(BasicBlock, 128, 2, stride=2)
        self.layer3 = self._make_layer(BasicBlock, 256, 2, stride=2)
        self.layer4 = self._make_layer(BasicBlock, 512, 2, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * BasicBlock.expansion, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )

    def _make_layer(self, block, out_channels, blocks, stride):
        downsample = None

        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion)
            )

        layers = [block(self.in_channels, out_channels, stride, downsample)]
        self.in_channels = out_channels * block.expansion

        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = self.fc(x)
        return x

In [7]:
# Initialize TensorBoard writers
current_time = time.strftime("%Y%m%d-%H%M%S")
adam_writer = SummaryWriter(f'runs/ResNet18_Adam_{current_time}')
sgd_writer = SummaryWriter(f'runs/ResNet18_SGD_{current_time}')

# Initialize device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [8]:
# Define two variants with different hyperparameters
variants = [
    {
        'name': 'ResNet18_Adam',
        'optimizer': 'adam',
        'lr': 0.001,
        'weight_decay': 0.0001,
        'scheduler': True,
        'writer': adam_writer
    },
    {
        'name': 'ResNet18_SGD',
        'optimizer': 'sgd',
        'lr': 0.01,
        'momentum': 0.9,
        'weight_decay': 0.001,
        'scheduler': True,
        'writer': sgd_writer
    }
]

In [9]:
def train_model(model, train_loader, val_loader, optimizer, criterion, scheduler, device, num_epochs, variant):
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    best_acc = 0.0
    best_model_wts = None

    for epoch in range(num_epochs):
        # Training
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for batch_idx, (inputs, targets) in enumerate(tqdm(train_loader, desc=f"{variant['name']} - Epoch {epoch+1}/{num_epochs} [Training]")):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            # TensorBoard batch metrics
            global_step = epoch * len(train_loader) + batch_idx
            variant['writer'].add_scalar('Batch/Train_Loss', loss.item(), global_step)
            variant['writer'].add_scalar('Batch/Train_Accuracy', 100. * correct / total, global_step)

        train_loss = running_loss / len(train_loader)
        train_acc = 100. * correct / total
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)

        # Validation (no report here)
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += targets.size(0)
                val_correct += predicted.eq(targets).sum().item()

        val_loss /= len(val_loader)
        val_acc = 100. * val_correct / val_total
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        # TensorBoard epoch metrics
        variant['writer'].add_scalar('Epoch/Train_Loss', train_loss, epoch)
        variant['writer'].add_scalar('Epoch/Train_Accuracy', train_acc, epoch)
        variant['writer'].add_scalar('Epoch/Val_Loss', val_loss, epoch)
        variant['writer'].add_scalar('Epoch/Val_Accuracy', val_acc, epoch)

        # Scheduler step
        if scheduler:
            scheduler.step(val_acc)

        print(f"{variant['name']} - Epoch {epoch+1}/{num_epochs}: "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = model.state_dict()
            torch.save(best_model_wts, f"best_{variant['name']}.pth")
            print(f"New best model saved with val accuracy: {best_acc:.2f}%")

    model.load_state_dict(best_model_wts)
    return model, history


In [10]:
def evaluate_model(model, loader, criterion, device, variant_name="model"):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for inputs, targets in tqdm(loader, desc=f"Evaluating {variant_name}"):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    loss = running_loss / len(loader)
    acc = 100. * correct / total

    # Confusion matrix and classification report
    cm = confusion_matrix(all_targets, all_preds)
    report = classification_report(all_targets, all_preds, target_names=train_dataset.classes)

    # Save confusion matrix
    cm_df = pd.DataFrame(cm, index=train_dataset.classes, columns=train_dataset.classes)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm_df, annot=True, cmap='Blues', fmt='g')
    plt.title(f'Confusion Matrix - {variant_name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    os.makedirs("confusion_matrices", exist_ok=True)
    filename = os.path.join("confusion_matrices", f"{variant_name}_confusion_matrix.png")
    plt.savefig(filename)
    plt.close()

    # Save report
    os.makedirs("reports", exist_ok=True)
    with open(f"reports/{variant_name}_classification_report.txt", "w") as f:
        f.write(report)

    print(f"Confusion matrix saved to {filename}")
    print("Classification Report:")
    print(report)

    return loss, acc, all_preds, all_targets

In [11]:
# Train for each variant

num_epochs = 10

for variant in variants:
    print(f"\nTraining with variant: {variant['name']}")
    
    model = ResNet18(num_classes=6).to(device)
    criterion = nn.CrossEntropyLoss()

    optimizer = (
        optim.Adam(model.parameters(), lr=variant['lr'], weight_decay=variant['weight_decay'])
        if variant['optimizer'] == 'adam'
        else optim.SGD(model.parameters(), lr=variant['lr'], momentum=variant['momentum'], weight_decay=variant['weight_decay'])
    )

    scheduler = (
        optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=3, verbose=True)
        if variant.get('scheduler')
        else None
    )

    # TensorBoard writer (separate for each run)
    log_dir = os.path.join("runs", variant['name'])
    os.makedirs(log_dir, exist_ok=True)
    variant['writer'] = SummaryWriter(log_dir=log_dir)

    # Train
    model, history = train_model(model, train_loader, val_loader, optimizer, criterion, scheduler, device, num_epochs, variant)

    # Final Evaluation
    print(f"\nFinal evaluation for variant: {variant['name']}")
    evaluate_model(model, val_loader, criterion, device, variant_name=variant['name'])

    variant['writer'].close()



Training with variant: ResNet18_Adam


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
ResNet18_Adam - Epoch 1/10 [Training]: 100%|██████████| 220/220 [00:25<00:00,  8.62it/s]


ResNet18_Adam - Epoch 1/10: Train Loss: 0.9409, Train Acc: 62.38% | Val Loss: 0.7794, Val Acc: 69.97%
New best model saved with val accuracy: 69.97%


ResNet18_Adam - Epoch 2/10 [Training]: 100%|██████████| 220/220 [00:26<00:00,  8.32it/s]


ResNet18_Adam - Epoch 2/10: Train Loss: 0.6958, Train Acc: 75.02% | Val Loss: 1.0838, Val Acc: 62.10%


...

ResNet18_SGD - Epoch 9/10: Train Loss: 0.3255, Train Acc: 88.40% | Val Loss: 0.4571, Val Acc: 84.40%


ResNet18_SGD - Epoch 10/10 [Training]: 100%|██████████| 220/220 [00:24<00:00,  8.84it/s]


ResNet18_SGD - Epoch 10/10: Train Loss: 0.3120, Train Acc: 89.02% | Val Loss: 0.6223, Val Acc: 79.10%

Final evaluation for variant: ResNet18_SGD


Evaluating ResNet18_SGD: 100%|██████████| 47/47 [00:02<00:00, 18.40it/s]


Confusion matrix saved to confusion_matrices/ResNet18_SGD_confusion_matrix.png
Classification Report:
              precision    recall  f1-score   support

   buildings       0.58      0.95      0.72       437
      forest       0.98      0.82      0.90       474
     glacier       0.76      0.83      0.80       553
    mountain       0.89      0.60      0.72       525
         sea       0.77      0.89      0.83       510
      street       0.93      0.65      0.76       501

    accuracy                           0.79      3000
   macro avg       0.82      0.79      0.79      3000
weighted avg       0.82      0.79      0.79      3000

